In [35]:
import os
import keras
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import  Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.callbacks import EarlyStopping, LearningRateScheduler
from random import shuffle
import math
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
from keras.models import load_model
from EEGModels import EEGNet  

In [36]:
PathSpectogramFolder='out'
OutputPath='model/3/path'
OutputPathModels='model/3/'
interictalSpectograms=[]
preictalSpectograms=[]  #This array contains syntetic data, it's created to have a balance dataset and it's used for training
preictalRealSpectograms=[]  #This array containt the real preictal data, it's used for testing
#patients = ["01"]
patients = ["01","02","03","05","09"]
nSeizure=0

In [37]:
def loadSpectogramData(indexPat):
    global interictalSpectograms
    global preictalSpectograms
    global preictalRealSpectograms
    global nSeizure
    f = open(PathSpectogramFolder+'/paz'+patients[indexPat]+'/legendAllData.txt', 'r')
    line=f.readline()
    while(not "SEIZURE" in line):
        line=f.readline()
    nSeizure=int(line.split(":")[1].strip())
    line=f.readline()
    line=f.readline()#legge il numero di spectogrammi. non lo salvo dato che non mi serve
    nSpectograms=int(line.strip())
    nFileForSeizure=math.ceil(math.ceil(nSpectograms/50)/nSeizure)
    line=f.readline()#leggo il percorso del primo file
    
    #Lettura path files Interictal
    cont=-1
    indFilePathRead=0
    while("npy" in line and indFilePathRead<nSeizure*nFileForSeizure):
        if(indFilePathRead%nFileForSeizure==0):
            interictalSpectograms.append([])
            cont=cont+1
            interictalSpectograms[cont].append(line.split(' ')[2].rstrip())#.rstrip() remove \n
            indFilePathRead=indFilePathRead+1
        else:
            if(len(line.split(' '))>=3):
                interictalSpectograms[cont].append(line.split(' ')[2].rstrip())
            indFilePathRead=indFilePathRead+1
            
        line=f.readline()
    line=f.readline()#leggo PREICTAL
    line=f.readline()#leggo n° spectogram
    line=f.readline()#leggo n°seizure(SEIZURE X)

    #Lettura path files Preictal
    cont=-1
    indFilePathRead=0   
    #while(line and indFilePathRead<nSeizure*nFileForSeizure):    
    while(line.strip()!=""):
        if("SEIZURE" in line):
            line=f.readline()#ho letto n°seizure(SEIZURE X) perciò scorro in avanti
            if(len(line.split(' '))>=3):
                preictalSpectograms.append([])
                cont=cont+1
                preictalSpectograms[cont].append(line.split(' ')[2].rstrip())
                indFilePathRead=indFilePathRead+1
        else:
            if(len(line.split(' '))>=3):
                preictalSpectograms[cont].append(line.split(' ')[2].rstrip())
            indFilePathRead=indFilePathRead+1
            
        line=f.readline()
        
    line=f.readline()#leggo REAL_PREICTAL
    line=f.readline()#leggo n° spectogram
    line=f.readline()#leggo n°seizure(SEIZURE X)

    #Lettura path files Real Preictal
    cont=-1
    while(line):
        if("SEIZURE" in line):
            line=f.readline()#ho letto n°seizure(SEIZURE X) perciò scorro in avanti
            preictalRealSpectograms.append([])
            cont=cont+1
            preictalRealSpectograms[cont].append(line.split(' ')[2].rstrip())
        else:
            preictalRealSpectograms[cont].append(line.split(' ')[2].rstrip())
            
        line=f.readline()
    f.close()

In [38]:
def getFilesPathWithoutSeizure(indexSeizure):
    filesPath=[]
    for i in range(0, nSeizure):
        if(i!=indexSeizure):
            filesPath.extend(interictalSpectograms[i])
            filesPath.extend(preictalSpectograms[i])
    shuffle(filesPath)
    return filesPath

In [39]:
def generate_arrays_for_training(paths, start=0, end=100):
    while True:
        from_=int(len(paths)/100*start)
        to_=int(len(paths)/100*end)
        for i in range(from_, int(to_)):
            f=paths[i]
            x = np.load(PathSpectogramFolder+f)
            x=np.array([x])
            x=x.swapaxes(0,1)
            x=x[0]
            if('P' in f):
                y = np.repeat([[0,1]], x.shape[0], axis=0)
            else:
                y = np.repeat([[1,0]], x.shape[0], axis=0)
            yield(x,y)

In [40]:
def generate_arrays_for_predict(indexPat, paths, start=0, end=100):
    print(paths)
    while True:
        f = open(PathSpectogramFolder+'/paz'+patients[indexPat]+'/legendAllData.txt', 'r')
        line=f.readline()
        while(not "SEIZURE" in line):
            line=f.readline()
        nSeizure=int(line.split(":")[1].strip())
        from_=int(len(paths)/100*start)
        to_=int(len(paths)/100*end)
        for j in 0, nSeizure:
            for i in range(from_, int(to_)):
                f=paths[j][i]
                x = np.load(PathSpectogramFolder+f)
                x=np.array([x])
                x=x.swapaxes(0,1)
                x=x[0]
                yield(x)

In [41]:
print("START")
if not os.path.exists(OutputPathModels):
    os.makedirs(OutputPathModels)
print("Parameters loaded")
nSeizure=0

for indexPat in range(0, len(patients)):
    print('Patient '+patients[indexPat])
    loadSpectogramData(indexPat) 
       
result='Out Seizure, True Positive, False Positive, False negative, Second of Inter in Test, Sensitivity, FPR \n'
filesPath = getFilesPathWithoutSeizure(indexPat)
    
initial_learning_rate = 0.001
decay_steps = 1000  # Numero di passi di addestramento dopo i quali applicare il decay
decay_rate = 0.1  # Tasso di decay
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate, staircase=True)

model = EEGNet(nb_classes = 2, Chans = 22, Samples = 114)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.Recall(name='sen')])
                
model.summary()

history = model.fit(generate_arrays_for_training(filesPath, end=75), #end=75),#It take the first 75%
                        validation_data=generate_arrays_for_training(filesPath, start=75),#start=75), #It take the last 25%
                        #steps_per_epoch=10000, epochs=10)
                        steps_per_epoch=int((len(filesPath)-int(len(filesPath)/100*25))),#*25), 
                        validation_steps=int((len(filesPath)-int(len(filesPath)/100*75))),#*75),
                        epochs=60, max_queue_size=2, shuffle=True, callbacks=[LearningRateScheduler(lr_schedule)])# 100 epochs è meglio #aggiungere criterio di stop in base accuratezza

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Creates a HDF5 file 
model.save(OutputPathModels+"ModelPat"+patients[indexPat]+"/"+'ModelOutSeizure'+str(i+1)+'.h5')
print("Model saved")
        
epochs = range(len(train_loss))

plt.figure()
plt.plot(epochs, train_loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()
plt.plot(epochs, train_acc, label='Training accuracy')
plt.plot(epochs, val_acc, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.show()

print('Training end')


START
Parameters loaded
Patient 01
Patient 02
Patient 03
Patient 05
Patient 09


ValueError: The `data_format` argument must be one of "channels_first", "channels_last". Received: channel_first